In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras 
from sklearn.preprocessing import OneHotEncoder as ONE
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split as tts
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from sklearn.metrics import classification_report

In [0]:
data = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = data.load_data()

In [0]:
#scaling
train_images = train_images/255
test_images = test_images/255

In [0]:
train_images = train_images.reshape(train_images.shape[0],train_images.shape[1],train_images.shape[2],1)
test_images = test_images.reshape(test_images.shape[0],test_images.shape[1],test_images.shape[2],1)

In [0]:
train_labels.resize(len(train_labels),1)
test_labels.resize(len(test_labels),1)

In [78]:
#one hot encoding 
ONE = ONE()
Y_train = ONE.fit_transform(train_labels)
Y_train = Y_train.toarray()
Y_test = ONE.fit_transform(test_labels)
Y_test = Y_test.toarray()


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [0]:
datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)

In [0]:
#initializing callbacks
ES = EarlyStopping(monitor='val_loss',mode='auto',patience=2,min_delta=.03)
LR = ReduceLROnPlateau(monitor='val_loss',mode='auto',min_lr=.001)
MC = ModelCheckpoint(filepath='best_model.hdf5',monitor='val_loss',save_best_only=True)

In [0]:
#Model 1 without batch normalization and using mini-batch grad descent,data_augmentation
model1 = keras.models.Sequential()

model1.add(keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu',input_shape=[28, 28, 1]))
model1.add(keras.layers.MaxPool2D((2,2)))

model1.add(keras.layers.Conv2D(64, (3, 3),activation='relu'))
model1.add(keras.layers.MaxPool2D((2,2)))

model1.add(keras.layers.Flatten())
model1.add(keras.layers.Dense(1024, activation='relu'))

model1.add(keras.layers.Dense(10, activation='softmax'))

In [0]:
model1.compile(keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [88]:
#model1.fit(train_images,Y_train, validation_split=.1, epochs=10, batch_size=64,callbacks=[ES,LR])
X_train,X_test,y_train,y_test = tts(train_images,Y_train, test_size=.1)
model1.fit_generator(datagen.flow(X_train, y_train, batch_size=100),epochs=10,steps_per_epoch=600,validation_data=(X_test,y_test))

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


600/600 [==============================] - 112s 187ms/step - loss: 1.1214 - acc: 0.6053 - val_loss: 0.7427 - val_acc: 0.7193
Epoch 2/10
600/600 [==============================] - 109s 182ms/step - loss: 0.7868 - acc: 0.7108 - val_loss: 0.7639 - val_acc: 0.7227
Epoch 3/10
600/600 [==============================] - 110s 183ms/step - loss: 0.7225 - acc: 0.7323 - val_loss: 0.7069 - val_acc: 0.7380
Epoch 4/10
600/600 [==============================] - 110s 183ms/step - loss: 0.6819 - acc: 0.7473 - val_loss: 0.6538 - val_acc: 0.7572
Epoch 5/10
600/600 [==============================] - 110s 183ms/step - loss: 0.6468 - acc: 0.7601 - val_loss: 0.6024 - val_acc: 0.7692
Epoch 6/10
600/600 [==============================] - 110s 183ms/step - loss: 0.6207 - acc: 0.7707 - val_loss: 0.5864 - val_acc: 0.7808
Epoch 7/10
600/600 [==============================] - 111s 185ms/step - loss: 0.5990 - acc: 0.7781 - val_loss: 0.5242 - val_acc: 0.8060
Epoch 8/10
600/600 [==============================] - 110s 

In [89]:
Evaluation1 = model1.evaluate(test_images,Y_test,batch_size=100)
print('Loss on augmented data without batch normalization, dropout and early-stopping:')
print('Test_loss:',Evaluation[0])
print('Test_accuracy:',Evaluation[1])

10000/10000 [==============================] - 4s 386us/step
[0.5052466577291489, 0.8216999995708466]


In [0]:
#with batch normalization and dropout
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=[28, 28, 1]))
model.add(keras.layers.BatchNormalization(momentum=.9,epsilon=.001))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(64, (3, 3)))
model.add(keras.layers.BatchNormalization(momentum=.9,epsilon=.001))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

In [91]:
#With batch normalization and dropout,data augmentation
model.compile(keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

#model.fit(train_images,Y_train, validation_split=.1, epochs=10, batch_size=100)

model.fit_generator(datagen.flow(X_train, y_train, batch_size=100),epochs=15,steps_per_epoch=600,validation_data=(X_test,y_test),callbacks=[ES,LR,MC])

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


600/600 [==============================] - 137s 229ms/step - loss: 1.7429 - acc: 0.3633 - val_loss: 1.0055 - val_acc: 0.6548
Epoch 2/15
600/600 [==============================] - 134s 224ms/step - loss: 1.2485 - acc: 0.5423 - val_loss: 0.8317 - val_acc: 0.7107
Epoch 3/15
600/600 [==============================] - 135s 224ms/step - loss: 1.0865 - acc: 0.6014 - val_loss: 0.7604 - val_acc: 0.7325
Epoch 4/15
600/600 [==============================] - 135s 224ms/step - loss: 0.9986 - acc: 0.6316 - val_loss: 0.7137 - val_acc: 0.7453
Epoch 5/15
600/600 [==============================] - 135s 224ms/step - loss: 0.9431 - acc: 0.6544 - val_loss: 0.6710 - val_acc: 0.7550
Epoch 6/15
600/600 [==============================] - 134s 224ms/step - loss: 0.8981 - acc: 0.6688 - val_loss: 0.6526 - val_acc: 0.7577
Epoch 7/15
600/600 [==============================] - 135s 225ms/step - loss: 0.8659 - acc: 0.6811 - val_loss: 0.6677 - val_acc: 0.7535


In [124]:
#Evaluation of accuracy
Evaluation = model.evaluate(test_images,Y_test,batch_size=100)
print('Loss on augmented data without batch normalization, dropout and early-stopping:')
print('Test_loss:',Evaluation[0])
print('Test_accuracy:',Evaluation[1])


10000/10000 [==============================] - 7s 663us/step
Loss on augmented data without batch normalization, dropout and early-stopping:
Test_loss: 0.668391487300396
Test_accuracy: 0.7508000022172928


In [126]:
# Classification report on both the models
from sklearn.preprocessing import OneHotEncoder as ON

Y_pred = model.predict_proba(test_images)
Y_pred1 = model1.predict_proba(test_images)

classes = np.argmax(Y_pred,axis=1).reshape(len(Y_pred),1)
classes1 = np.argmax(Y_pred1,axis=1).reshape(len(Y_pred),1)

ON = ON()
classes = ON.fit_transform(classes).toarray() 
classes1 = ON.fit_transform(classes1).toarray() 

print('Classification report on augmented data with batch_norm, early_stopping and dropout')
print(classification_report(Y_test,classes))


print('Classification report on augmented data without batch_norm, early_stopping and dropout')
print(classification_report(Y_test,classes1))


Classification report on augmented data with batch_norm, early_stopping and dropout
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1000
           1       0.93      0.96      0.95      1000
           2       0.48      0.83      0.61      1000
           3       0.72      0.82      0.77      1000
           4       0.67      0.51      0.58      1000
           5       0.77      0.94      0.85      1000
           6       0.39      0.10      0.16      1000
           7       0.83      0.80      0.81      1000
           8       0.93      0.92      0.92      1000
           9       0.94      0.86      0.90      1000

   micro avg       0.75      0.75      0.75     10000
   macro avg       0.74      0.75      0.73     10000
weighted avg       0.74      0.75      0.73     10000
 samples avg       0.75      0.75      0.75     10000

Classification report on augmented data without batch_norm, early_stopping and dropout
              pr

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [0]:
print(classification_report(Y_test,classes))